# 최종 모델 답변 생성

In [1]:
import torch
from langchain_community.vectorstores import Chroma
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI


RAG_PROMPT_TEMPLATE = (
"""
절대 한국어로만 답변하십시오. 영어를 사용하지 마십시오.  
You must always respond in Korean. Do not use English.  

당신은 대한민국 현행 법령을 기반으로 답변하는 RAG 챗봇입니다.  
아래 지침을 따르고, 명확하고 간결한 답변을 제공하세요.

- 질문이 단순하면 한 문장으로 답변하세요.  
- 질문이 복잡하면 2~3문장 이내로 답변하세요.  
- 법 조항을 직접 나열하지 말고, 질문과 관련된 핵심 내용을 요약하여 자연스럽게 설명하세요.  
- 각 문장에 해당하는 법적 근거 조항을 괄호 안에 정확히 명시하세요. (예: 도로교통법 제12조 제1항)  
- 질문을 반복하지 말고, 불필요한 형식적인 표현을 포함하지 마세요.  
- **정확도, 확률, 신뢰도 같은 수치를 답변에 절대 포함하지 마세요.**  
- Context 내에서만 답변을 생성하고, 모르는 내용은 `"제가 제공할 수 있는 정보가 없습니다."`라고만 답변하세요.  

{question}에 대한 답변을 다음 Context를 참고하여 작성하세요.

Context:  
{context}  

답변:
"""
)

SYS_PROMPT_TEMPLATE = (
"""
당신은 대한민국 현행 법령을 기반으로 작동하는 RAG 챗봇입니다.  
절대 한국어로만 답변하십시오. 영어를 사용하지 마십시오.  

답변을 생성할 때, 다음 원칙을 따르세요.

1. **간결한 답변 제공**  
   - 단순한 질문 → 한 문장  
   - 복잡한 질문 → 2~3문장  
   - 불필요한 설명, 배경 정보, 반복 표현 금지  

2. **법 조항 요약 방식**  
   - 법 조항을 직접 나열하지 않고 질문과 관련된 핵심 내용을 요약  
   - 각 문장에 해당하는 법적 근거 조항을 괄호 안에 명확히 표기 (예: 도로교통법 제17조 제2항)

3. **불필요한 정보 제한**  
   - "정확도", "신뢰도", "확률" 등의 수치는 절대 포함 금지  
   - "또한", "추가로", "이 외에도"와 같은 확장 표현 사용 금지  

4. **추론 및 임의 해석 금지**  
   - 제공된 Context 범위 내에서만 답변 작성  
   - 필요한 정보가 없으면 `"제가 제공할 수 있는 정보가 없습니다."`라고만 답변  
"""
)

DEVICE = "cpu"

class SentenceTransformerEmbedding:
    def __init__(self, model_name):
        self.model = SentenceTransformer(model_name, trust_remote_code=True, device=DEVICE)

    def embed_documents(self, texts):
        return self.model.encode(texts, convert_to_tensor=True).cpu().tolist()

    def embed_query(self, text):
        return self.model.encode([text], convert_to_tensor=True).cpu().tolist()[0]


class ReRanker:
    def __init__(self, model_name):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        self.reranker = AutoModelForSequenceClassification.from_pretrained(model_name, trust_remote_code=True).to("mps" if torch.backends.mps.is_available() else "cpu")

    def re_rank(self, query, docs):
        if not docs:
            return []

        inputs = self.tokenizer([(query, doc) for doc in docs], padding=True, truncation=True, return_tensors='pt').to("mps" if torch.backends.mps.is_available() else "cpu")

        with torch.no_grad():
            outputs = self.reranker(**inputs)
            scores = outputs.logits.squeeze(-1).cpu()
        
        return [doc for _, doc in sorted(zip(scores, docs), reverse=True)]


def load_chroma_retriever_with_embedding_and_reranker(chroma_dir, embedding_model_name="jinaai/jina-embeddings-v3", reranker_model_name="BAAI/bge-reranker-base", k=10):
    embedding_model = SentenceTransformerEmbedding(embedding_model_name)
    vectorstore = Chroma(persist_directory=chroma_dir, embedding_function=embedding_model)
    retriever = vectorstore.as_retriever(search_kwargs={'k': k})
    reranker = ReRanker(reranker_model_name)
    return retriever, reranker


def setup_rag_system_with_reranker(llm_base_url):
    llm = ChatOpenAI(
        base_url=llm_base_url,
        api_key="lm-studio",
        model="teddylee777/EEVE-Korean-Instruct-10.8B-v1.0-gguf",
        temperature=0.1,
    )
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", SYS_PROMPT_TEMPLATE),
        ("user", RAG_PROMPT_TEMPLATE),
    ])
    rag_chain = prompt_template | llm | StrOutputParser()
    return llm, rag_chain


def run_pipeline(question, llm_base_url="http://localhost:1234/v1", chroma_dir="chroma_data_folder_final_copy", reranker_model_name="BAAI/bge-reranker-base"):
    if not question.strip():
        return "질문을 입력해주세요.", [], []

    retriever, reranker = load_chroma_retriever_with_embedding_and_reranker(
        chroma_dir=chroma_dir, reranker_model_name=reranker_model_name
    )
    retrieved_docs = retriever.invoke(question)
    if not retrieved_docs:
        return "제가 제공할 수 있는 정보가 없습니다.", [], []

    ranked_docs = reranker.re_rank(question, [doc.page_content for doc in retrieved_docs])
    if not ranked_docs:
        return "제가 제공할 수 있는 정보가 없습니다.", retrieved_docs, []

    context = " ".join(ranked_docs)
    llm, rag_chain = setup_rag_system_with_reranker(llm_base_url)
    response = rag_chain.invoke({"question": question, "context": context})
    return response, retrieved_docs, ranked_docs


끝


In [2]:
import pandas as pd

simple_question1 = pd.read_csv('score_check/question/simple_questions_1.csv')
simple_question2 = pd.read_csv('score_check/question/simple_questions_2.csv')
complex_question1 = pd.read_csv('score_check/question/complex_questions_1.csv')
complex_question2 = pd.read_csv('score_check/question/complex_questions_2.csv')

final_question = pd.concat([simple_question1, simple_question2, complex_question1, complex_question2], ignore_index=True)
final_question

In [ ]:
final_answer = []

for i in final_question['질문']:
    response, retrieved_docs, ranked_docs = run_pipeline(i)

    final_answer.append({
        "질문": i,
        "내 모델 답변": response,
        "검색된 문서": retrieved_docs,
        "재정렬된 문서": ranked_docs
    })

In [ ]:
final_pd = pd.DataFrame(final_answer)